## Setup

In [1]:
import os
import sys
import random

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import numpy as np

sys.path.append("../src")

import models  # noqa: E402
import utils  # noqa: E402
import train.backprop  # noqa: E402
import train.ff  # noqa: E402
import train.pc  # noqa: E402

os.makedirs("../results", exist_ok=True)

In [2]:
# Config

batch_size = 32
n_classes = 10
lr = 0.001
n_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
# Load data

train_loader, val_loader, test_loader = utils.load_mnist_data(batch_size)
class_names = [str(i) for i in range(n_classes)]

## Backprop

In [8]:
import importlib

importlib.reload(train.backprop)

# Initialize model, loss, optimizer
model = models.LeNet5(n_classes=n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

n_params = sum(p.numel() for p in model.parameters())
print(f"{torchinfo.summary(model, input_size=(batch_size, 1, 32, 32))}\n")

# Train model using backpropagation
history = train.backprop.backprop(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    n_epochs=n_epochs,
    device=device,
)

# Save trained model
save_path = "../results/backprop-model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"\nTraining completed. Model saved to '{save_path}'")

# Save training history
dst = "../results/backprop-history.csv"
pd.DataFrame(history).to_csv(dst, index=False)

# Evaluate model
test_metrics = train.backprop.evaluate(model, test_loader, criterion, device)
utils.save_metrics(test_metrics, "../results/mnist-metrics.csv", "backprop")

# Final summary
print("\nBackpropagation – Training Summary")
print("-" * 60)
print(f"Train accuracy: {history['train_accuracies'][-1]:.2f}")
print(f"Val accuracy: {history['val_accuracies'][-1]:.2f}")
print(f"Test accuracy: {test_metrics['accuracy']:.2f}")
print(f"Params: {n_params:,}")
print(f"Epochs: {n_epochs}")

Layer (type:depth-idx)                   Output Shape              Param #
LeNet5                                   [32, 10]                  --
├─Conv2d: 1-1                            [32, 6, 28, 28]           156
├─AvgPool2d: 1-2                         [32, 6, 14, 14]           --
├─Conv2d: 1-3                            [32, 16, 10, 10]          2,416
├─AvgPool2d: 1-4                         [32, 16, 5, 5]            --
├─Conv2d: 1-5                            [32, 120, 1, 1]           48,120
├─Linear: 1-6                            [32, 84]                  10,164
├─Linear: 1-7                            [32, 10]                  850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 13.54
Input size (MB): 0.13
Forward/backward pass size (MB): 1.67
Params size (MB): 0.25
Estimated Total Size (MB): 2.05

Starting backpropagation training
------------------------------------------------------------


KeyboardInterrupt: 

## Forward-forward

In [6]:
import importlib

importlib.reload(train.ff)

# Initialize model, loss, optimizer
model = models.FFLeNet5(n_classes=n_classes)
optimizers = [torch.optim.Adam(layer.parameters(), lr=lr) for layer in model.layers]
from torchvision.models.feature_extraction import get_graph_node_names

print(f"{get_graph_node_names(model)=}")

n_params = sum(p.numel() for p in model.parameters())
print(f"{torchinfo.summary(model, input_size=(batch_size, 1 + n_classes, 32, 32))}\n")

# Train model using forward-forward
# - No criterion: FF does not use global loss
# - No optimizer: FF uses per-layer optimizers
history = train.ff.forward_forward(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizers=optimizers,
    n_epochs=n_epochs,
    device=device,
    n_classes=n_classes,
    threshold=2.0,
)

# Save trained model
save_path = "../results/ff-model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"\nTraining completed. Model saved to '{save_path}'")

# Save training history
dst = "../results/ff-history.csv"
pd.DataFrame(history).to_csv(dst, index=False)

# Evaluate model
test_metrics = train.ff.evaluate(model, test_loader, n_classes, device)
utils.save_metrics(test_metrics, "../results/mnist-metrics.csv", "ff")

# Final summary
print("\nForward-Forward – Training Summary")
print("-" * 60)
if history["train_accuracies"][-1] is not None:
    print(f"Train accuracy: {history['train_accuracies'][-1]:.2f}")
print(f"Val accuracy: {history['val_accuracies'][-1]:.2f}")
print(f"Test accuracy: {test_metrics['accuracy']:.2f}")
print(f"Params: {n_params:,}")
print(f"Epochs: {n_epochs}")

get_graph_node_names(model)=(['x', 'conv1.conv1_weight', 'conv1.conv1_bias', 'conv1.conv2d', 'conv1.relu', 'pool1', 'conv2.conv2_weight', 'conv2.conv2_bias', 'conv2.conv2d', 'conv2.relu', 'pool2', 'conv3.conv3_weight', 'conv3.conv3_bias', 'conv3.conv2d', 'conv3.relu', 'flatten', 'fc1.fc1_weight', 'fc1.fc1_bias', 'fc1.linear', 'fc1.relu', 'fc2.fc2_weight', 'fc2.fc2_bias', 'fc2.linear'], ['x', 'conv1.conv1_weight', 'conv1.conv1_bias', 'conv1.conv2d', 'conv1.relu', 'pool1', 'conv2.conv2_weight', 'conv2.conv2_bias', 'conv2.conv2d', 'conv2.relu', 'pool2', 'conv3.conv3_weight', 'conv3.conv3_bias', 'conv3.conv2d', 'conv3.relu', 'flatten', 'fc1.fc1_weight', 'fc1.fc1_bias', 'fc1.linear', 'fc1.relu', 'fc2.fc2_weight', 'fc2.fc2_bias', 'fc2.linear'])
Layer (type:depth-idx)                   Output Shape              Param #
FFLeNet5                                 [32, 10]                  --
├─FFConv2d: 1-1                          [32, 6, 28, 28]           1,656
├─AvgPool2d: 1-2                 

## Predictive Coding

In [7]:
import importlib

importlib.reload(train.pc)

# Hyperparameters
n_inference_steps = 5
inference_lr = 5e-5

# Initialize model, loss, optimizer
model = models.PCLeNet5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

n_params = sum(p.numel() for p in model.parameters())
print(f"{torchinfo.summary(model, input_size=(batch_size, 1, 32, 32))}\n")

history = train.pc.predictive_coding(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    n_epochs,
    device,
    inference_lr=inference_lr,
    n_inference_steps=n_inference_steps,
)

# Save trained model
save_path = "../results/pc-model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(model.state_dict(), save_path)
print(f"\nTraining completed. Model saved to '{save_path}'")

# Save training history
dst = "../results/pc-history.csv"
pd.DataFrame(history).to_csv(dst, index=False)

# Evaluate model
test_metrics = train.pc.evaluate(model, test_loader, criterion, device)
utils.save_metrics(test_metrics, "../results/mnist-metrics.csv", "pc")

# Final summary
print("\Predictive Coding – Training Summary")
print("-" * 60)
print(f"Train accuracy: {history['train_accuracies'][-1]:.2f}")
print(f"Val accuracy: {history['val_accuracies'][-1]:.2f}")
print(f"Test accuracy: {test_metrics['accuracy']:.2f}")
print(f"Params: {n_params:,}")
print(f"Epochs: {n_epochs}")

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [32, 10]                  --
├─Sequential: 1-1                        [32, 6, 14, 14]           --
│    └─Conv2d: 2-1                       [32, 6, 28, 28]           156
│    └─ReLU: 2-2                         [32, 6, 28, 28]           --
│    └─MaxPool2d: 2-3                    [32, 6, 14, 14]           --
├─Sequential: 1-2                        [32, 16, 5, 5]            --
│    └─Conv2d: 2-4                       [32, 16, 10, 10]          2,416
│    └─ReLU: 2-5                         [32, 16, 10, 10]          --
│    └─MaxPool2d: 2-6                    [32, 16, 5, 5]            --
├─Sequential: 1-3                        [32, 120]                 --
│    └─Flatten: 2-7                      [32, 400]                 --
│    └─Linear: 2-8                       [32, 120]                 48,120
│    └─ReLU: 2-9                         [32, 120]                 --
├─Seque

KeyboardInterrupt: 

## Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


def plot_learning_curve_and_performance(
    metrics: pd.DataFrame,
    histories: dict[str, pd.DataFrame],
    chance: float,
    dataset_name: str = "MNIST",
    save_path: str = None,
):
    keys = list(histories.keys())
    colors = sns.color_palette("deep", len(keys))

    # Infer number of epochs from first method
    epochs = max(len(df) for df in histories.values())

    # Get per-method stats
    times = [df["epoch_times"].mean() for df in histories.values()]
    accuracies = [metrics[metrics["model"] == k]["accuracy"].values[0] for k in keys]
    accuracies = [a * 100 for a in accuracies]  # Convert to percentage

    # Plot layout
    fig = plt.figure(figsize=(13, 6))
    fig.suptitle(f"Performance Comparison – {dataset_name}", weight="bold", y=1.0)
    widths = [0.6, 0.2, 0.2]
    spec = fig.add_gridspec(ncols=3, nrows=2, width_ratios=widths)

    train_ax = fig.add_subplot(spec[0, 0])
    valid_ax = fig.add_subplot(spec[1, 0])
    time_ax = fig.add_subplot(spec[:, 1])
    acc_ax = fig.add_subplot(spec[:, 2])

    fig.subplots_adjust(wspace=0.24, hspace=0.36, top=0.85, bottom=0.05)
    x_vals = np.arange(1, epochs + 1)

    # Get ylim based on max and min losses
    ylim = (
        -0.05,
        max(
            max(df["train_losses"].max() for df in histories.values()),
            max(df["val_losses"].max() for df in histories.values()),
        )
        + 0.05,
    )

    # Training subplot
    # train_ax.set_xticks(x_vals[::2])
    train_ax.set_xticks(x_vals)
    train_ax.set_xlim(1, epochs)
    train_ax.set_xlabel("Epoch")
    train_ax.set_ylabel("Loss")
    train_ax.set_ylim(ylim)
    train_ax.set_title("Training")
    acc_train = train_ax.twinx()
    acc_train.spines[["right"]].set_visible(True)
    acc_train.axhline(chance, ls="dashed", color="k", alpha=0.7)
    acc_train.set_ylim(-5, 105)
    acc_train.set_ylabel("Accuracy (%)")
    acc_train.yaxis.label.set_position((1.08, 0.5))

    # Validation subplot
    # valid_ax.set_xticks(x_vals[::2])
    valid_ax.set_xticks(x_vals)
    valid_ax.set_xlim(1, epochs)
    valid_ax.set_xlabel("Epoch")
    valid_ax.set_ylabel("Loss")
    valid_ax.set_ylim(ylim)
    valid_ax.set_title("Validation")
    acc_valid = valid_ax.twinx()
    acc_valid.spines[["right"]].set_visible(True)
    acc_valid.axhline(chance, ls="dashed", color="k", alpha=0.7)
    acc_valid.set_ylim(-5, 105)
    acc_valid.set_ylabel("Accuracy (%)")
    acc_valid.yaxis.label.set_position((1.08, 0.5))

    # Curves
    lw = 2
    all_lines = []
    all_algorithms = []
    for df, k, color in zip(histories.values(), keys, colors):
        l1 = train_ax.plot(x_vals, df["train_losses"], label=k, color=color, lw=lw)[0]
        acc_train.plot(
            x_vals,
            df["train_accuracies"] * 100,
            label=k,
            color=color,
            ls="dashed",
            lw=lw,
        )
        valid_ax.plot(x_vals, df["val_losses"], label=k, color=color, lw=lw)
        acc_valid.plot(
            x_vals, df["val_accuracies"] * 100, label=k, color=color, ls="dashed", lw=lw
        )
        all_lines.append(l1)
        all_algorithms.append(k)

    # Accuracy bar
    acc_ax.bar(np.arange(len(accuracies)), height=accuracies, color=colors, label=keys)
    acc_ax.yaxis.set_label_position("right")
    acc_ax.yaxis.tick_right()
    acc_ax.set_ylabel("Accuracy (%)")
    acc_ax.spines[["top", "left"]].set_visible(False)
    acc_ax.set_title("Test Accuracy")
    acc_ax.set_xticks(np.arange(len(keys)))
    acc_ax.set_xticklabels(keys)
    for i, v in enumerate(accuracies):
        acc_ax.text(
            i, v + 1, f"{v:.1f}%", ha="center", va="bottom", fontsize=9, color="black"
        )

    # Time per epoch bar
    time_ax.bar(np.arange(len(times)), height=times, color=colors, label=keys)
    time_ax.yaxis.set_label_position("right")
    time_ax.yaxis.tick_right()
    time_ax.set_ylabel("Time (s)")
    time_ax.spines[["top", "left"]].set_visible(False)
    time_ax.set_title("Time per Epoch")
    time_ax.set_xticks(np.arange(len(keys)))
    time_ax.set_xticklabels(keys)
    for i, v in enumerate(times):
        time_ax.text(
            i, v + 0.1, f"{v:.2f}s", ha="center", va="bottom", fontsize=9, color="black"
        )

    # Global legend on top
    fig.legend(
        all_lines,
        all_algorithms,
        loc="upper center",
        bbox_to_anchor=(0.5, 0.97),  # center above plot
        ncol=len(all_algorithms),
        frameon=False,
    )

    # Remove individual legends
    train_ax.legend_.remove() if train_ax.legend_ else None
    valid_ax.legend_.remove() if valid_ax.legend_ else None
    time_ax.legend_.remove() if time_ax.legend_ else None

    if save_path:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

In [ ]:
metrics = pd.read_csv("../results/mnist-metrics.csv")
histories = {
    "backprop": pd.read_csv("../results/backprop-history.csv"),
    "ff": pd.read_csv("../results/ff-history.csv"),
    "pc": pd.read_csv("../results/pc-history.csv"),
}

plot_learning_curve_and_performance(
    metrics=metrics,
    histories=histories,
    save_path="../results/training-curves.png",
    chance=100 / n_classes,
)

# TODO: per class accuracy